# Dog Breed Identification: DLNN Final Project
Ian Battin & Liam Schmid




In this project, we are tackling the Kaggle Competition: [Dog Breed Identification](https://www.kaggle.com/c/dog-breed-identification/).

In this competition, you are given a dataset containing images of 120 different breeds of dogs. It is our job to classify images in the test set into these 120 different breeds.

Being an image based challenge, CNNs are the obvious choice. We will discuss the different strategies and optimizations we tried as well as provide all the code used at each step.

## Preparing the data
The data is provided in two folders - train and test - where each file is an image. There is a labels.csv file which maps an image filename to a label (breed).

To prepare our data for use, we needed to move all of the images in the training set into seperate folders grouped by breed:

This function takes a labels file in the format provided by Kaggle and outputs a mapping of photo ids to their breed.

In [19]:
import sys
import os
import shutil
from keras import models
from keras.preprocessing import image
from keras import backend
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt


def generateIDMapping(labelsFile):
    idMap = {}

    with open(labelsFile) as f:
        for line in f:
            if line.rstrip() != 0 and line.rstrip() != "id,breed":
                id, breed = line.rstrip().split(
                    ',')[0], line.rstrip().split(',')[1]

                idMap[id] = breed

    return idMap

The following functions create a train directory, validation directory, and test directory. For the train and validation directories, it generates subfolders for each image type to work with fit_generator.

In [20]:
def makeTrainDirectories(idMap, source, dest):
    if os.path.exists(dest):
        shutil.rmtree(dest)

    for filename in os.listdir(source):
        id = filename.split('.')[0]
        breed = idMap[id]

        srcPath = source+"/"+filename
        destPath = dest+"/"+breed+"/"+filename

        path = Path(dest+"/"+breed)
        path.mkdir(parents=True, exist_ok=True)
        shutil.copyfile(srcPath, destPath)


def makeValidationDirectories():
    for dirname in os.listdir('train'):
        samples = np.asarray(os.listdir('train/'+dirname))
        valSamples = np.random.choice(samples, len(samples)//4, replace=False)

        if os.path.exists('validation/'+dirname):
            shutil.rmtree('validation/'+dirname)
        for sample in valSamples:
            path = Path('validation/'+dirname)
            path.mkdir(parents=True, exist_ok=True)

            shutil.move('train/'+dirname+'/'+sample,
                        'validation/'+dirname+'/'+sample)


def makeTestDirectory(source, dest):
    if os.path.exists(dest):
        shutil.rmtree(dest)

    dest = dest + "/images"
    path = Path(dest)
    path.mkdir(parents=True, exist_ok=True)

    for filename in os.listdir(source):
        srcPath = source+"/"+filename
        destPath = dest+"/"+filename
        shutil.copyfile(srcPath, destPath)

In [21]:
idMap = generateIDMapping('data/labels.csv')

makeTrainDirectories(idMap, 'data/train', 'train')
makeValidationDirectories()
makeTestDirectory('data/test', 'test')

## Training the model


## Looking at predictions
Let's look at the top 5 predictions for example images.

<img src="data/train/0ac12f840df2b15d46622e244501a88c.jpg">

## Analyzing the model
Let's get a better sense of how inception is looking at the images. We will look at th channel outputs for several images.

First, we'll get a tensor representation of our sample image.

First, we get the activation model, or a model which returns the outputs of the included layers.
<img src='data/train/a21caab32c00011d38f1e409fbf65d19.jpg'>

In [22]:
def get_img_tensor(img_path, display_img=False):
    img = image.load_img(img_path, target_size=(299, 299))
    img_tensor = image.img_to_array(img)
    img_tensor = np.expand_dims(img_tensor, axis=0)
    img_tensor /= 255.

    if display_img:
        plt.imshow(img_tensor[0])
        plt.show()

    return img_tensor

img_tensor = get_img_tensor('data/train/a21caab32c00011d38f1e409fbf65d19.jpg')

Next, we get the activation model, or a model which returns the outputs of the included layers.

In [ ]:
model = models.load_model('Model46.h5')

inception = model.layers[0]

# Returns model which returns output of first n_layers given an image.
def get_activation_model(inception_model, n_layers=294):
    layer_outputs = [
        layer.output for layer in inception_model.layers[1:n_layers]]

    activation_model = models.Model(
        inputs=inception_model.layers[0].input, outputs=layer_outputs)

    return activation_model

activation_model = get_activation_model(inception)

Now we call predict from our activation model. This yields us the outputs from each layer for this image. We organize each channels output for a given layer in a grid and display it with matplotlib.

In [ ]:
# Scale output values to be between 0,255 to display
def scale_img_values(channel_img):
    channel_img -= channel_img.mean()
    channel_img /= channel_img.std()
    channel_img *= 64
    channel_img += 128
    channel_img = np.clip(channel_img, 0, 255).astype('uint8')

    return channel_img

def analyze_channels_for_img(inception, activation_model, img_tensor,
                             n_layers=294, imgs_per_row=16, n_activations=16):
    activations = activation_model.predict(img_tensor)

    first_layer_activation = activations[0]

    layer_names = [layer.name for layer in inception.layers[1:n_layers]]

    for layer_name, activation in zip(layer_names[:n_activations],
                                      activations[:n_activations]):
        if 'batch_normalization' in layer_name:
            continue

        # Channels in output
        n_channels = activation.shape[-1]

        img_size = activation.shape[1]
        # Columns to display in grid
        n_cols = n_channels // imgs_per_row

        display_grid = np.zeros((img_size * n_cols, imgs_per_row * img_size))

        for col in range(n_cols):
            for row in range(imgs_per_row):
                channel_img = scale_img_values(
                    activation[0, :, :, col * imgs_per_row + row])

                display_grid[col * img_size: (col + 1) * img_size,
                             row * img_size: (row + 1) * img_size] = channel_img

        scale = 1. / img_size
        plt.figure(figsize=(scale * display_grid.shape[1],
                            scale * display_grid.shape[0]))
        plt.title(layer_name)
        plt.grid(False)
        plt.imshow(display_grid, aspect='auto', cmap='viridis')

    plt.show()

analyze_channels_for_img(inception, activation_model, img_tensor)

Lets compare for this new image.
<img src='data/train/0ec9be8b32f2b9eff2b817a7f722b118.jpg'>

In [ ]:
img_tensor = get_img_tensor('data/train/0ec9be8b32f2b9eff2b817a7f722b118.jpg')

analyze_channels_for_img(inception, activation_model, img_tensor)